##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# 이미지 로드

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/images"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org 에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub)에서 소스보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운로드</a>
  </td>
</table>

이 튜토리얼은 `tf.data`를 사용하는 이미지 데이터세트를 불러오는 방법의 간단한 예시를 제공합니다.

이러한 예시에서 사용된 데이터세트는 데이터세트마다 이미지의 하나의 클래스르 가지고 이미지의 디렉토리로 분배됩니다.

## 설치

In [0]:
import tensorflow as tf

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
tf.__version__

### 이미지 검색

트레이닝을 시작하기 전에 네트워크에 인식하고 싶은 새로운 클래스에 대해 학습하기 위한 이미지세트가 필요합니다. 구글에서 꽃 이미지를 허가하는 크리에이티브 커먼즈에 대한 기록물을 사용할 수 있습니다.

참고: 모든 이미지에는 CC-BY 라이센스가 부여되며, 생성자는`LICENSE.txt`파일에 나열됩니다.

In [0]:
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

다운로드(218MB)후, 꽃 이미지를 복사할 수 있습니다.

디렉토리에는 클래스마다 5개으 서브 디렉토리가 있습니다.

In [0]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

각 디렉토리에는 그 타입의 꽃 이미지가 포함되어 있습니다. 이것은 장미입니다.

In [0]:
roses = list(data_dir.glob('roses/*'))

for image_path in roses[:3]:
    display.display(Image.open(str(image_path)))

## keras.preprocessing을 사용하여 로드

이미지를 로드하는 간단한 방법은 `tf.keras.preprocessing`를 사용하는 것입니다.

In [0]:
# 1./255는 [0,1] 범위에서 unit8에서 float32로 변환합니다.
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

로더의 매개변수를 정의합니다.

In [0]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [0]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

배치를 점검합니다.

In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [0]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

## tf.data를 사용하여 로드

위의 `keras.preprocessing`방법은 편리하지만 세가지 결점이 있습니다. 

1. 늦습니다. 밑의 성과 부분을 참고합니다.
2. 섬세한 컨트롤이 결여되어 있습니다.
3. TensorFlow의 나머지 부분과 잘 통합되어 있지 않습니다.

파일을 `tf.data.dataset`로 로드하려면 데이터세트에 먼저 파일 경로의 데이터세트를 생성해야합니다.

In [0]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [0]:
for f in list_ds.take(5):
  print(f.numpy())

파일 경로를 (`이미지, 레이블`) 쌍으로 변환하는 짧은 pure-tensorflow 함수를 사용합니다.

In [0]:
def get_label(file_path):
  # 경로를 경로 구성요소 목록으로 변환합니다.
  parts = tf.strings.split(file_path, os.path.sep)
  # 마지막 두 번째 클래스 디렉터리입니다.
  return parts[-2] == CLASS_NAMES

In [0]:
def decode_img(img):
  # 압축된 문자열을 3D unit8 텐서로 변환합니다.
  img = tf.image.decode_jpeg(img, channels=3)
  # [0,1]범위의 float으로 변환하려면 convert_image_dtype을 사용합니다.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # 이미지 크기를 원하는 크기로 조정합니다.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [0]:
def process_path(file_path):
  label = get_label(file_path)
  # 파일에서 데이터를 문자열로 로드합니다.
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

Dataset.map을 사용하여 `이미지, 레이블` 쌍의 데이터세트를 만듭니다.

In [0]:
# 다중 영상이 병렬로 로드/처리되도록 num_parallel_calls로 설정합니다.
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [0]:
for image, label in labeled_ds.take(1):
  print("이미지 모양: ", image.numpy().shape)
  print("레이블: ", label.numpy())

### 기본 훈련법

이 데이터세트에서 모델을 트레이닝하려면 다음이 요구됩니다.

* 잘 섞습니다.
* 일괄 처리합니다.
* 최대한 빨리 이용할 수 있는 배치를 합니다.

이 기능은 `tf.data` api를 사용하고 쉽게 추가할 수 있습니다.

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # 이것은 작은 데이터세트로, 한 번만 로드하고 메모리에 보관합니다.
  # 메모리에 맞지 않는 데이터세트의 사전 처리 작업을 캐싱하려면 .cache(filename)을 
  # 사용합니다.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # 계속 반복합니다.
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch`는 모델이 훈련하느 동안 데이터세트가 백그라운드에 배치를 가져올 수
  # 있도록 합니다.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [0]:
train_ds = prepare_for_training(labeled_ds)

image_batch, label_batch = next(iter(train_ds))

In [0]:
show_batch(image_batch.numpy(), label_batch.numpy())

## 성과

주의: 이 부분에서는 성과에 도움이되는 간단한 기술을 몇가지 소개하겠습니다. 상세한 가이드에 대해서는 [Input Pipeline Performance](../../guide/performance/datasets) 를 참고합니다.

먼저 데이터세트의 성과를 체크하는 기능을 살펴보겠습니다.

In [0]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

두 데이터 생성기의 속도를 비교해 봅니다.

In [0]:
# `keras.preprocessing`을 이용했을 때
timeit(train_data_gen)

In [0]:
# `tf.data`을 이용했을 때
timeit(train_ds)

성과 향상의 대부분은 `.cache`의 사용에서 발생합니다.

In [0]:
uncached_ds = prepare_for_training(labeled_ds, cache=False)
timeit(uncached_ds)

데이터세트가 메모리에 맞지 않는 경우에는 캐시파일을 사용해 몇 가지 이점을 유지합니다.

In [0]:
filecache_ds = prepare_for_training(labeled_ds, cache="./flowers.tfcache")
timeit(filecache_ds)